Preliminary work:
    - count 0s, 0.1s, and missing values

In [1]:
import synapseclient
import pandas as pd
import numpy as np
import json

syn = synapseclient.Synapse()
syn.login(silent=True)

overall_scores = 'syn25575156'
overall_scores = syn.tableQuery(f"select * from {overall_scores}").asDataFrame()
# overall_scores = pd.read_json(overall_scores.path, orient='records')

overall_scores.columns

Index(['GeneName', 'ENSG', 'Overall', 'Overall_rank', 'GeneticsScore',
       'OmicsScore', 'LiteratureScore', 'FlyNeuroPathScore',
       'SM_Druggability_bucket', 'safety_bucket', 'feasibility_bucket',
       'ABability_bucket', 'new_modality_bucket', 'tissue_engagement_bucket',
       'Pharos_class', 'classification', 'safety_bucket_definition',
       'feasibility_bucket_definition', 'ABability_bucket_definition',
       'new_modality_bucket_definition', 'tissue_engagement_bucket_definition',
       'isScored_genetics', 'isScored_omics', 'isScored_lit',
       'isScored_neuropath'],
      dtype='object')

Getting some information on how to deal with Overall Scores when they're 0.

In [2]:
interesting_ensembls = overall_scores[interesting_columns+scored]
print("Missing Overall Values: " + str(interesting_ensembls[interesting_ensembls['Overall'].isna()].shape[0]))
print("Overall == 0: " + str(interesting_ensembls[interesting_ensembls['Overall']==0].shape[0]))
zeros = interesting_ensembls[interesting_ensembls['Overall']==0]
print("Overall == 0 AND any of the isScored columns == 'Y': " + str(zeros[zeros.isin(['Y']).any(axis=1)].shape[0]))

NameError: name 'interesting_columns' is not defined

It turns out whenever they're 0, at least one of the isScored columns is 'Y'.  We don't have to exclude any of them, but I'll add it to the calculation fuction in case that case arises in the future.  Here I create a mapping of the columns and their corresponding isScored flag:

In [ ]:
interesting_columns = ['ENSG', 'Overall', 'GeneticsScore', 'OmicsScore', 'LiteratureScore']
scored = ['isScored_genetics', 'isScored_omics', 'isScored_lit']
mapping = dict(zip(interesting_columns[2:], scored))
mapping['Overall'] = None
mapping

Function to calculate distribution.  We should see a warning when running this function that we're trying to modify a copy of a dataframe.  This is exactly what we want to do in this case.

In [ ]:
def calculate_distribution(df, col, is_scored = None):

    if is_scored is not None:
        df = df[df[is_scored]=='Y']
    else:
        df = df[df.isin(['Y']).any(axis=1)]

    if df[col].dtype == object:
        df[col] = df[col].astype(float)
    
    obj = {}
    
    obj["distribution"] = list(pd.cut(df[col], bins=10, precision=3).value_counts())
    obj["min"] = np.around(df[col].min(), 4)
    obj["max"] = np.around(df[col].max(), 4)
    obj["mean"] = np.around(df[col].mean(), 4)
    discard, obj["bins"] = list(pd.cut(df[col], bins=10, precision=3, retbins=True))
    obj["bins"] = np.around(obj["bins"].tolist()[1:], 2)
    base = [0, *obj["bins"][:-1]]
    obj["bins"] = zip(base, obj["bins"])
    obj["bins"] = list(obj["bins"])
    obj["first_quartile"] = np.around(df[col].quantile(q=0.25, interpolation='midpoint'))
    obj["third_quartile"] = np.around(df[col].quantile(q=0.75, interpolation='midpoint'))
    
    return obj

In [ ]:
neo_matrix = {}
for col in interesting_columns[1:]: # excludes the ENSG
    neo_matrix[col] = calculate_distribution(overall_scores, col, mapping[col])

In [ ]:
additional_data = [{'name': 'Overall Score', 'syn_id': 'syn25913473', 'wiki_id': '613107'},
                    {'name': 'Genetics Score', 'syn_id': 'syn25913473', 'wiki_id': '613104'},
                    {'name': 'Genomics Score', 'syn_id': 'syn25913473', 'wiki_id': '613106'},
                    {'name': 'Literature Score', 'syn_id': 'syn25913473', 'wiki_id': '613105'}
                  ]
for col, additional in zip(neo_matrix.keys(), additional_data):
    neo_matrix[col]['name'] = additional['name']
    neo_matrix[col]['syn_id'] = additional['syn_id']
    neo_matrix[col]['wiki_id'] = additional['wiki_id']
    
print(neo_matrix)


In [ ]:
with open('../output/distribution_data.json', 'w') as outfile:
    json.dump(neo_matrix, outfile)